In [1]:
import numpy as np
from mlp import DenseLayer, MLP

def mse(y_hat: np.ndarray, y: np.ndarray) -> np.float64:
    return np.mean(np.square(y_hat - y))

In [2]:
# Input data
rows = 5
cols = 1
input_vector = np.random.normal(size=(rows,))
dummy_pred_vector = np.random.normal(size=(rows,))

In [3]:
# Test loss
type(mse(dummy_pred_vector, input_vector))

numpy.float64

In [4]:
# Instantiate Layer
lyr = DenseLayer(input_dims=rows, num_units=2)

In [5]:
lyr(input_vector).shape

(2,)

In [6]:
model = MLP(input_dims=input_vector.shape[0], hidden_units=2, targets=1, loss_function=mse, learning_rate=1e-3)

In [7]:
# Result of forward pass
fwd = model._forward_pass(inputs=input_vector)
print(fwd)
print(fwd.shape)

[0.01610959]
(1,)
